# Practice3 (test_submission1_4, 2_4.csv)
> Public Score
> - test_submission1_3: 25136.09206
> - test_submission2_3: 28583.13469

In [129]:
import sys, os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# column 보기 30개로 설정
pd.set_option('display.max_columns', 30)

In [130]:
# 데이터 경로 설정
file_path = os.path.join("c:\\Users\\prohe\\OneDrive\\바탕 화면\\알고리즘PBL2\\", 'data')
train_origin = pd.read_csv(file_path+'\dataset.csv')
test_origin = pd.read_csv(file_path+'\problem.csv')

In [131]:
train_origin.shape, test_origin.shape

((1340, 24), (130, 23))

In [132]:
# 전처리를 위한 원본 데이터 복사
train = train_origin.copy()
test = test_origin.copy()

In [133]:
# 메모리를 효율적으로 사용하기 위한 downcast 함수 정의
def downcast(df, verbose=True):     # verbose 옵션 추가: (True)인 경우 몇 퍼센트 압축됐는지 출력
    start_mem = df.memory_usage().sum() / 1024**2   # 초기 메모리 사용량
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2

    if verbose:
        print(f'{(100*(start_mem - end_mem) / start_mem):.1f}% 압축됨')

    return df

In [134]:
downcast(train)
downcast(test)

67.2% 압축됨
68.7% 압축됨


,Id,LotArea,Street,LotConfig,OverallQual,OverallCond,YearBuilt,YearRemodAdd,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageType,GarageYrBlt,GarageCars,GarageArea,YrSold
0,1341,11200,Pave,Inside,5,5,1965,1965,1040,1040,0,1040,1,0,3,1,5,0,Detchd,1965,1,384,2008
1,1342,7200,Pave,Corner,5,7,1951,2000,900,900,0,900,1,0,3,1,5,0,Detchd,2005,2,576,2010
2,1343,16905,Pave,Inside,5,6,1959,1959,1350,1328,0,1328,1,1,2,1,5,2,Attchd,1959,1,308,2007
3,1344,9180,Pave,CulDSac,5,7,1983,1983,840,884,0,884,1,0,2,1,5,0,Attchd,1983,2,504,2007
4,1345,7200,Pave,Inside,5,7,1920,1996,530,581,530,1111,1,0,3,1,6,0,Detchd,1935,1,288,2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,1466,11478,Pave,Inside,8,5,2007,2008,1704,1704,0,1704,2,0,3,1,7,1,Attchd,2008,3,772,2010
126,1467,16321,Pave,CulDSac,5,6,1957,1997,1484,1600,0,1600,1,0,2,1,6,2,Attchd,1957,1,319,2006
127,1468,6324,Pave,Inside,4,6,1927,1950,520,520,0,520,1,0,1,1,4,0,Detchd,1920,1,240,2008
128,1469,8500,Pave,Inside,4,4,1920,1950,649,649,668,1317,1,0,3,1,6,0,Detchd,1920,1,250,2008


# 전처리

### 결측치 처리

In [135]:
# 결측치는 확인해보면 주차장이 모두 없는 경우 이므로 0으로 대체
train['GarageType'].fillna('Nan', inplace=True)
train['GarageYrBlt'].fillna(0, inplace=True)

In [136]:
train.isnull().sum().sum(), test.isnull().sum().sum()

(0, 0)

In [137]:
train = train[train.GrLivArea < 4500].reset_index(drop=True)
train['SalePrice'] = np.log1p(train['SalePrice'])
y = train['SalePrice'].reset_index(drop=True)

In [138]:
train_features = train.drop(['SalePrice'], axis=1)
test_features = test
features = pd.concat([train_features, test_features]).reset_index(drop=True)

In [139]:
features.shape

(1468, 23)

In [140]:
features.drop('Id', axis=1, inplace=True)

In [141]:
features.drop('YrSold', axis=1, inplace=True)

In [142]:
objects = []
for i in features.columns:
    if features[i].dtype == object:
        objects.append(i)
#features.update(features[objects].fillna('None'))  
print(objects)

['Street', 'LotConfig', 'GarageType']


In [143]:
numeric_dtypes = ['int8', 'int16', 'int32', 'int64', 'float8', 'float16', 'float32', 'float64']
numerics = []
for i in features.columns:
    if features[i].dtype in numeric_dtypes:
        numerics.append(i)
#features.update(features[numerics].fillna(0))
numerics

['LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'GrLivArea',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea']

In [144]:
features

,LotArea,Street,LotConfig,OverallQual,OverallCond,YearBuilt,YearRemodAdd,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageType,GarageYrBlt,GarageCars,GarageArea
0,8450,Pave,Inside,7,5,2003,2003,856,856,854,1710,2,1,3,1,8,0,Attchd,2003.0,2,548
1,9600,Pave,FR2,6,8,1976,1976,1262,1262,0,1262,2,0,3,1,6,1,Attchd,1976.0,2,460
2,11250,Pave,Inside,7,5,2001,2002,920,920,866,1786,2,1,3,1,6,1,Attchd,2001.0,2,608
3,9550,Pave,Corner,7,5,1915,1970,756,961,756,1717,1,0,3,1,7,1,Detchd,1998.0,3,642
4,14260,Pave,FR2,8,5,2000,2000,1145,1145,1053,2198,2,1,4,1,9,1,Attchd,2000.0,3,836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463,11478,Pave,Inside,8,5,2007,2008,1704,1704,0,1704,2,0,3,1,7,1,Attchd,2008.0,3,772
1464,16321,Pave,CulDSac,5,6,1957,1997,1484,1600,0,1600,1,0,2,1,6,2,Attchd,1957.0,1,319
1465,6324,Pave,Inside,4,6,1927,1950,520,520,0,520,1,0,1,1,4,0,Detchd,1920.0,1,240
1466,8500,Pave,Inside,4,4,1920,1950,649,649,668,1317,1,0,3,1,6,0,Detchd,1920.0,1,250


In [145]:
from scipy.stats import skew, boxcox_normmax
from scipy.special import boxcox1p

skew_features = features[numerics].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.8]
skew_index = high_skew.index

for i in skew_index:
    features[i] = boxcox1p(features[i], boxcox_normmax(features[i] + 1))   # boxcox_normmax: 최적의 람다값 찾기

c:\Users\prohe\anaconda3\envs\py38\lib\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
c:\Users\prohe\anaconda3\envs\py38\lib\site-packages\scipy\stats\_stats_py.py:4461: NearConstantInputWarning: An input array is nearly constant; the computed correlation coefficient may be inaccurate.
  warnings.warn(stats.NearConstantInputWarning(msg))


In [146]:
features = features.drop(['Street'], axis=1)
features['YrBltAndRemod']=features['YearBuilt']+features['YearRemodAdd']
features['TotalSF']=features['TotalBsmtSF'] + features['1stFlrSF'] + features['2ndFlrSF']
features['Total_Bathrooms'] = features['FullBath'] + (0.5 * features['HalfBath'])

features['Has2ndfloor'] = features['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
features['Hasgarage'] = features['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
features['Hasbsmt'] = features['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
features['Hasfireplace'] = features['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

In [147]:
features.shape

(1468, 27)

In [148]:
features

,LotArea,LotConfig,OverallQual,OverallCond,YearBuilt,YearRemodAdd,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageType,GarageYrBlt,GarageCars,GarageArea,YrBltAndRemod,TotalSF,Total_Bathrooms,Has2ndfloor,Hasgarage,Hasbsmt,Hasfireplace
0,8.909157,Inside,7,5,2003,2003,856,6.550012,1240.298106,10.015644,2,1,3,0.717329,8,0,Attchd,2003.0,2,548,4006,2102.848119,2.5,1,1,1,0
1,9.032982,FR2,6,8,1976,1976,1262,6.914050,0.000000,9.486786,2,0,3,0.717329,6,1,Attchd,1976.0,2,460,3952,1268.914050,2.0,0,1,1,1
2,9.186829,Inside,7,5,2001,2002,920,6.617717,1258.850450,10.092357,2,1,3,0.717329,6,1,Attchd,2001.0,2,608,4003,2185.468167,2.5,1,1,1,1
3,9.027915,Corner,7,5,1915,1970,756,6.658639,1089.436248,10.022840,1,0,3,0.717329,7,1,Detchd,1998.0,3,642,3885,1852.094887,1.0,1,1,1,1
4,9.416661,FR2,8,5,2000,2000,1145,6.822917,1549.975643,10.462078,2,1,4,0.717329,9,1,Attchd,2000.0,3,836,4000,2701.798560,2.5,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463,9.206285,Inside,8,5,2007,2008,1704,7.194853,0.000000,10.009454,2,0,3,0.717329,7,1,Attchd,2008.0,3,772,4015,1711.194853,2.0,0,1,1,1
1464,9.547444,CulDSac,5,6,1957,1997,1484,7.136022,0.000000,9.898840,1,0,2,0.717329,6,2,Attchd,1957.0,1,319,3954,1491.136022,1.0,0,1,1,1
1465,8.627723,Inside,4,6,1927,1950,520,6.080941,0.000000,8.012228,1,0,1,0.717329,4,0,Detchd,1920.0,1,240,3877,526.080941,1.0,0,1,1,0
1466,8.914883,Inside,4,4,1920,1950,649,6.289706,955.035138,9.560306,1,0,3,0.717329,6,0,Detchd,1920.0,1,250,3870,1610.324844,1.0,1,1,1,0


In [149]:
# test
features.drop(['GrLivArea', 'FullBath', 'HalfBath', 'GarageArea'], axis=1, inplace=True)

In [150]:
final_features = pd.get_dummies(features).reset_index(drop=True)
final_features.shape

(1468, 33)

In [151]:
X = final_features.iloc[:len(y), :]
X_test = final_features.iloc[len(y):, :]
X.shape, y.shape, X_test.shape

((1338, 33), (1338,), (130, 33))

In [152]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error

kfolds = KFold(n_splits=10, shuffle=True, random_state=42)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model, X=X):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kfolds))
    return (rmse)

In [153]:
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

In [154]:
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler


ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=alphas_alt, cv=kfolds))
lasso = make_pipeline(RobustScaler(), LassoCV(max_iter=10000000, alphas=alphas2, random_state=42, cv=kfolds))
elasticnet = make_pipeline(RobustScaler(), ElasticNetCV(max_iter=10000000, alphas=e_alphas, cv=kfolds, l1_ratio=e_l1ratio))                                
svr = make_pipeline(RobustScaler(), SVR(C= 20, epsilon= 0.008, gamma=0.0003,))

In [155]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, loss='huber', random_state =42)  

In [156]:
from xgboost import XGBRegressor

xgboost = XGBRegressor(learning_rate=0.01,n_estimators=3460, max_depth=3, min_child_weight=0,
                       gamma=0, subsample=0.7, colsample_bytree=0.7, nthread=-1,
                       scale_pos_weight=1, seed=27, reg_alpha=0.00006)

In [157]:
from lightgbm import LGBMRegressor

lightgbm = LGBMRegressor(objective='regression', num_leaves=4, learning_rate=0.01, n_estimators=5000,
                         max_bin=200, bagging_fraction=0.75, bagging_freq=5, bagging_seed=7,
                         feature_fraction=0.2, feature_fraction_seed=7, verbose=-1,)

In [158]:
from mlxtend.regressor import StackingCVRegressor

stack_gen = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, gbr, xgboost, lightgbm),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)

In [159]:
score1 = cv_rmse(ridge , X)
print("Ridge: {:.4f} ({:.4f})\n".format(score1.mean(), score1.std()), )

score2 = cv_rmse(lasso , X)
print("LASSO: {:.4f} ({:.4f})\n".format(score2.mean(), score2.std()), )

score3 = cv_rmse(elasticnet)
print("Elastic net: {:.4f} ({:.4f})\n".format(score3.mean(), score3.std()), )

score4 = cv_rmse(svr)
print("SVR: {:.4f} ({:.4f})\n".format(score4.mean(), score4.std()), )

score5 = cv_rmse(gbr)
print("gbr: {:.4f} ({:.4f})\n".format(score5.mean(), score5.std()), )

score6 = cv_rmse(xgboost)
print("xgboost: {:.4f} ({:.4f})\n".format(score6.mean(), score6.std()), )

score7 = cv_rmse(lightgbm)
print("lightgbm: {:.4f} ({:.4f})\n".format(score7.mean(), score7.std()), )

Ridge: 0.1281 (0.0190)

LASSO: 0.1277 (0.0192)

Elastic net: 0.1276 (0.0192)

SVR: 0.1277 (0.0195)

gbr: 0.1309 (0.0152)

xgboost: 0.1293 (0.0163)

lightgbm: 0.1286 (0.0157)



In [160]:
model_performances = pd.DataFrame({
    'Model': ['Ridge', 'Lasso', 'ElasticNet', 'SVR', 'GradientBoosting', 'XGBoost', 'LightGBM'],
    'Score': [score1.mean(), score2.mean(), score3.mean(), score4.mean(), score5.mean(), score6.mean(), score7.mean()],
    'Std': [score1.std(), score2.std(), score3.std(), score4.std(), score5.std(), score6.std(), score7.std()]})

model_performances

,Model,Score,Std
0,Ridge,0.128100,0.018953
1,Lasso,0.127678,0.019157
2,ElasticNet,0.127650,0.019165
3,SVR,0.127660,0.019515
4,GradientBoosting,0.130945,0.015230
5,XGBoost,0.129300,0.016292
6,LightGBM,0.128640,0.015699


In [161]:
print('START Fit')

print('stack_gen')
stack_gen_model = stack_gen.fit(np.array(X), np.array(y))

print('elasticnet')
elastic_model_full_data = elasticnet.fit(X, y)

print('Lasso')
lasso_model_full_data = lasso.fit(X, y)

print('Ridge')
ridge_model_full_data = ridge.fit(X, y)

print('Svr')
svr_model_full_data = svr.fit(X, y)

print('GradientBoosting')
gbr_model_full_data = gbr.fit(X, y)

print('xgboost')
xgb_model_full_data = xgboost.fit(X, y)

print('lightgbm')
lgb_model_full_data = lightgbm.fit(X, y)

START Fit
stack_gen
elasticnet
Lasso
Ridge
Svr
GradientBoosting
xgboost
lightgbm


In [164]:
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor

param_lst = {
    'n_estimators' : [100, 300, 500, 1000, 1300, 1600],
    'learning_rate' : [0.0001, 0.001, 0.01, 0.1],
    'l2_leaf_reg' : [0.001, 0.01, 0.1],
    'random_strength' : [0.25, 0.5 ,1],
    'max_depth' : [3, 6, 9],
    'min_child_samples' : [2, 5, 10, 15, 20],
    'rsm' : [0.5, 0.7, 0.9],
    
}
cb = CatBoostRegressor(loss_function='RMSE', logging_level='Silent')
catboost = RandomizedSearchCV(estimator = cb, param_distributions = param_lst,
                              n_iter = 10, scoring = 'neg_root_mean_squared_error',
                              cv = 5)

catboost_search = catboost.fit(X, y)

KeyboardInterrupt: 

In [ ]:
print('Ridge RMSLE score on train data:')
print(rmsle(np.expm1(y), np.expm1(catboost_search.predict(X))))

In [162]:
print('Ridge RMSLE score on train data:')
print(rmsle(np.expm1(y), np.expm1(ridge_model_full_data.predict(X))))

print('Lasso RMSLE score on train data:')
print(rmsle(np.expm1(y), np.expm1(lasso_model_full_data.predict(X))))

print('ElasticNet RMSLE score on train data:')
print(rmsle(np.expm1(y), np.expm1(elastic_model_full_data.predict(X))))

print('SVR RMSLE score on train data:')
print(rmsle(np.expm1(y), np.expm1(svr_model_full_data.predict(X))))

print('GradientBoosting RMSLE score on train data:')
print(rmsle(np.expm1(y), np.expm1(gbr_model_full_data.predict(X))))

print('Xgboost RMSLE score on train data:')
print(rmsle(np.expm1(y), np.expm1(xgb_model_full_data.predict(X))))

print('lightgbm RMSLE score on train data:')
print(rmsle(np.expm1(y), np.expm1(lgb_model_full_data.predict(X))))

Ridge RMSLE score on train data:
24118.42387913953
Lasso RMSLE score on train data:
23987.743508339154
ElasticNet RMSLE score on train data:
23980.12228617623
SVR RMSLE score on train data:
24160.576189721385
GradientBoosting RMSLE score on train data:
9520.22565072458
Xgboost RMSLE score on train data:
12680.458796711684
lightgbm RMSLE score on train data:
18049.458556272635


In [165]:
print(rmsle(np.expm1(y), np.expm1(stack_gen_model.predict(np.array(X)))))

13485.864290476055


In [166]:
def blend_models_predict(X):
    return ((0.2 * elastic_model_full_data.predict(X)) + \
            (0.2 * lasso_model_full_data.predict(X)) + \
            (0.2 * ridge_model_full_data.predict(X)) + \
            (0.2 * svr_model_full_data.predict(X)) + \
            (0.05 * gbr_model_full_data.predict(X)) + \
            (0.05 * xgb_model_full_data.predict(X)) + \
            (0.05 * lgb_model_full_data.predict(X)) + \
            (0.05 * stack_gen_model.predict(np.array(X))))

In [167]:
print('RMSLE score on train data:')
print(rmsle(y, blend_models_predict(X)))

RMSLE score on train data:
0.11315161082857872


In [168]:
print('RMSLE score on train data:')
print(rmsle(np.expm1(y), np.expm1(blend_models_predict(X))))

RMSLE score on train data:
21341.45259516609


In [169]:
print('Predict submission')
submission_origin = pd.read_csv(file_path+'\submission.csv')
submission = submission_origin.copy()
submission.iloc[:,1] = (np.expm1(blend_models_predict(X_test)))

submission.to_csv('Day2_13.csv', index=False)

Predict submission
